# Track Characteristics - Practice Session Extraction

Test the practice extraction methodology using 2025 data.

**Why this notebook:**
- Validates extraction works with FP1/FP2 data (not just qualifying)
- Compares practice vs qualifying characteristics (track evolution)
- Tests the workflow we'll use in 2026

**For 2026:** Change `season = 2026` and this notebook extracts from actual 2026 practice sessions.

**The workflow (2026):**
1. Pre-weekend: Use 2025 baseline for initial predictions
2. Friday FP2: Run this notebook to extract 2026 characteristics
3. Post-practice: Use fresh 2026 data for quali predictions
4. Save: Update 2026 track database progressively

## Setup

In [1]:
import fastf1 as ff1
import pandas as pd
import numpy as np
from pathlib import Path
import json

import logging
logging.getLogger("fastf1").setLevel(logging.ERROR)

import sys

PROJECT_ROOT = Path.cwd().parents[0]
sys.path.append(str(PROJECT_ROOT))

from src.helpers.track_extraction import (
    extract_track_profile,
    identify_street_circuits,
    calculate_track_z_scores,
    describe_track_profile
)

ff1.Cache.enable_cache('../data/raw/.fastf1_cache')

import warnings
warnings.filterwarnings('ignore')

## Extract From Practice Sessions

Using FP2 is recommended over FP1:
- More laps, better data quality
- Track has evolved (more grip)
- Teams run different programs (long runs, quali sims)

But if FP2 has issues (rain, red flags), FP1 works too.

In [2]:
# For testing: use 2025 data
# For 2026: change this to 2026
season = 2025

event_name = 'Bahrain Grand Prix'
session_type = 'FP2'  # or 'FP1' if FP2 had issues

print(f"Extracting {event_name} - {season} {session_type}")
print(f"(Testing methodology - for 2026, change season variable above)")
print("=" * 60)

# Load practice session
session = ff1.get_session(season, event_name, session_type)
session.load(telemetry=True, laps=True)

print(f"🟢 Loaded {session.event['EventName']} {session_type}")
print(f"   Laps: {len(session.laps)}")
print(f"   Drivers: {len(session.laps['Driver'].unique())}")

Extracting Bahrain Grand Prix - 2025 FP2
(Testing methodology - for 2026, change season variable above)
🟢 Loaded Bahrain Grand Prix FP2
   Laps: 522
   Drivers: 20


### Extract characteristics

In [3]:
# Extract track profile from practice
profile = extract_track_profile(season, session)

if profile:
    print(f"🟢 Extracted track profile for {profile['track_name']}")
    print(f"\nKey metrics:")
    print(f"  Corner density: {profile.get('corner_density', 'N/A'):.2f} corners/km")
    print(f"  Min corner speed: {profile.get('min_corner_speed_kmh', 'N/A'):.0f} km/h")
    print(f"  Avg speed loss: {profile.get('avg_speed_loss_kmh', 'N/A'):.1f} km/h")
    print(f"  Full throttle: {profile.get('full_throttle_pct', 'N/A'):.1%}")
    print(f"  Top speed: {profile.get('top_speed_kmh', 'N/A'):.0f} km/h")
else:
    print("🔴 Failed to extract profile - check session data quality")

🟢 Extracted track profile for Bahrain Grand Prix

Key metrics:
  Corner density: 1.49 corners/km
  Min corner speed: 67 km/h
  Avg speed loss: 164.9 km/h
  Full throttle: 60.7%
  Top speed: 319 km/h


## Compare Practice vs Qualifying

Track conditions evolve during the weekend.

**Chronological order:** FP2 (Friday) → Qualifying (Saturday)

**Expected changes:**
- Track gets faster (more grip, rubber laid down)
- Teams optimize setup between sessions
- Qualifying typically faster than practice

In [4]:
# Load qualifying baseline (from notebook 05)
baseline_path = Path(f'../data/processed/testing_files/track_characteristics/{season}_track_characteristics.json')

if baseline_path.exists():
    with open(baseline_path, 'r') as f:
        baseline_data = json.load(f)
    
    if event_name in baseline_data['tracks']:
        baseline = baseline_data['tracks'][event_name]
        
        print(f"Comparison: {event_name} ({season})")
        print("=" * 60)
        print(f"{'Metric':<25} {session_type:<15} {'Qualifying':<15} {'Change':<10}")
        print("-" * 60)
        
        metrics_to_compare = [
            ('corner_density', 'corners/km', 2),
            ('min_corner_speed_kmh', 'km/h', 0),
            ('avg_speed_loss_kmh', 'km/h', 1),
            ('heavy_braking_pct', '%', 1),
            ('full_throttle_pct', '%', 1),
            ('top_speed_kmh', 'km/h', 0),
        ]
        
        for metric, unit, decimals in metrics_to_compare:
            if metric in baseline and metric in profile:
                practice_val = profile[metric]
                quali_val = baseline[metric]
                
                # Convert percentages for display
                if unit == '%':
                    quali_val *= 100
                    practice_val *= 100
                
                change = quali_val - practice_val
                pct_change = (change / practice_val * 100) if practice_val != 0 else 0
                
                practice_str = f"{practice_val:.{decimals}f} {unit}"
                quali_str = f"{quali_val:.{decimals}f} {unit}"
                change_str = f"{change:+.{decimals}f} ({pct_change:+.1f}%)"
                
                print(f"{metric:<25} {practice_str:<15} {quali_str:<15} {change_str:<10}")
        
        print("\nNote: Small differences expected - track evolves during weekend")
    else:
        print(f"⚠️  {event_name} not found in qualifying baseline")
        print(f"   Run notebook 05 first to create {season} baseline")
else:
    print(f"⚠️  {season} baseline not found - run notebook 05 first")

Comparison: Bahrain Grand Prix (2025)
Metric                    FP2             Qualifying      Change    
------------------------------------------------------------
corner_density            1.49 corners/km 1.49 corners/km +0.00 (+0.2%)
min_corner_speed_kmh      67 km/h         67 km/h         +0 (+0.0%)
avg_speed_loss_kmh        164.9 km/h      163.0 km/h      -1.9 (-1.1%)
heavy_braking_pct         87.5 %          87.5 %          +0.0 (+0.0%)
full_throttle_pct         60.7 %          59.6 %          -1.1 (-1.8%)
top_speed_kmh             319 km/h        315 km/h        -4 (-1.3%)

Note: Small differences expected - track evolves during weekend


## Save Practice Data

For 2026, this saves to the 2026 database.
For testing (2025), this validates the extraction works.

In [5]:
# Decide output file based on season
if season >= 2026:
    # Real 2026 data - save to 2026 database
    output_file = f'{season}_track_characteristics.json'
    print(f"Saving to {season} database (actual season data)")
else:
    # Testing with historical data - save separately
    output_file = f'{season}_track_characteristics_practice.json'
    print(f"Saving to practice test file (testing methodology with {season} data)")

output_path = Path(f'../data/processed/testing_files/track_characteristics/{output_file}')

# Load or create database
if output_path.exists():
    with open(output_path, 'r') as f:
        track_data = json.load(f)
    print(f"🟢 Loaded existing database ({len(track_data.get('tracks', {}))} tracks)")
else:
    track_data = {
        'metadata': {
            'season': season,
            'extracted_from_sessions': [session_type],
            'last_updated': None
        },
        'tracks': {}
    }
    print("🟢 Created new database")

# Add this track
if profile:
    track_data['tracks'][event_name] = profile
    track_data['metadata']['last_updated'] = pd.Timestamp.now().isoformat()
    
    # Save
    output_path.parent.mkdir(parents=True, exist_ok=True)
    with open(output_path, 'w') as f:
        json.dump(track_data, f, indent=2)
    
    print(f"\n🟢 Saved to {output_path}")
    print(f"  Total tracks: {len(track_data['tracks'])}")
    print(f"  Updated: {event_name}")
else:
    print("🔴 No profile to save - extraction failed")

Saving to practice test file (testing methodology with 2025 data)
🟢 Loaded existing database (1 tracks)

🟢 Saved to ../data/processed/testing_files/track_characteristics/2025_track_characteristics_practice.json
  Total tracks: 1
  Updated: Bahrain Grand Prix


## Summary

**Session order (normal weekend):** FP1 → FP2 → FP3 → Qualifying → Race<br>
**Session order (sprint weekend):** FP1 → Sprint Qualifying → Sprint → Qualifying → Race

**For testing (2025):**
- Validates practice extraction methodology works
- Compares FP2 → Qualifying (chronological order)
- Saves to separate practice file

**For 2026 (March onwards):**
1. Set `season = 2026` at top of notebook
2. Friday morning: Make pre-weekend predictions using 2025 baseline
3. Friday afternoon: Run this notebook after FP2 to extract 2026 characteristics
4. Friday evening: Use fresh 2026 data for quali predictions
5. Repeat for each race weekend
6. This notebook (as well as the others in this folder) will be productionalized once the whole logic is ready.